<a href="https://colab.research.google.com/github/dxspimentel/PUC/blob/main/MVP_Diego_Pimentel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **MVP Análise de Dados e Boas Práticas**

Nome: Diego Pimentel <br>
Matrícula: 4052025000258 <br>
Dataset: **Escolas Conectadas**

<br>
--- OBSERVAÇÃO IMPORTANTE ---

Este é um Dataset adaptado, devido a Segurança da Informação e Sigilo da Instituição.

------

# Descrição do Problema

Dados relacionados ao projeto *Escolas Conectadas*, cujo objetivo é levar internet a escolas com estrutura precária, ou sem estrutura, nas regiões mais afastadas das grandes cidades.<br>
Neste trabalho será desenvolvida uma solução que facilite a identificação de quais escolas foram de fato entregues, sem pendências, dentro do universo de escolas com obras já contratadas.

## Hipóteses do Problema

Cada escola tem a sua particularidade, com isso trabalha com escopo específico.
O escopo de cada projeto é dividido em 3 tipos de obras:<br>
1) Rede Interna<br>
2) Rede Externa<br>
3) Gerador Solar<br>

A questão é analisar se as etapas previstas em cada escola foram concluídas e com isso classificar como **Conectada**.

## Tipo de Problema

Este é um problema de **classificação supervisionada**.<br>
Cada etapa do projeto é classificada de acordo com o status de execução de obra. Uma vez que o escopo esteja totalmente atendido, a escola é classificada como entregue (**Conectada**).

## Seleção de Dados

Para este trabalho considerei duas bases de dados. A primeira com o monitoramento das obras, por projeto, e a segunda informando o escopo a ser considerado para cada escola.<br><br>

A base de projetos está verticalizada, ou seja, os status das etapas estão empilhados. Já a base de escopo já está bem definida, trazendo os escopos em 3 colunas distintas, para cada projeto.

# Importação das Bibliotecas Necessárias e Carga de Dados

Esta seção consolida todas as importações de bibliotecas necessárias para a análise, visualização e pré-processamento dos dados, bem como o carregamento inicial das bases para montar o dataset **Escolas Conectadas**.

In [42]:
# Bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# URLs
github_projetos = 'https://raw.githubusercontent.com/dxspimentel/PUC/main/PUC_Db_01_Base_Projetos.csv'
github_escopo = 'https://raw.githubusercontent.com/dxspimentel/PUC/main/PUC_Db_02_Base_Escopo.csv'

In [43]:
# Carregamento da Base de Projetos
df_projetos = pd.read_csv(github_projetos, encoding='latin-1', sep=';')
print('df_projetos carregado com sucesso')

df_projetos carregado com sucesso


In [44]:
# Carregamento da Base de Escopos
df_escopo = pd.read_csv(github_escopo, encoding='latin-1', sep=';')
print('df_escopo carregado com sucesso')

df_escopo carregado com sucesso


In [45]:
df_projetos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27418 entries, 0 to 27417
Data columns (total 7 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   CODCT                        27418 non-null  int64 
 1   PROJETO                      27418 non-null  object
 2   Obra                         27418 non-null  object
 3   Tipo Obra                    27418 non-null  object
 4   RDO                          27418 non-null  object
 5   STATUS_RDO                   27418 non-null  object
 6   DATA_VALIDACAO_SALA_TECNICA  20735 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.5+ MB


In [46]:
df_projetos.head()

/usr/local/lib/python3.11/dist-packages/google/colab/_dataframe_summarizer.py:88: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  cast_date_col = pd.to_datetime(column, errors="coerce")


,CODCT,PROJETO,Obra,Tipo Obra,RDO,STATUS_RDO,DATA_VALIDACAO_SALA_TECNICA
0,1,12345678,3-INSTALAÇÃO_DE_ACESSO,Rede de Acesso,RDO-00000000011,Cancelado,NaN
1,4,13029533,3-INSTALAÇÃO_DE_ACESSO,Rede de Acesso,RDO-00000000001,Cancelado,NaN
2,4,13031392,3-INSTALAÇÃO_DE_ACESSO,Rede de Acesso,RDO-00000000001,Cancelado,NaN
3,4,13029355,3-INSTALAÇÃO_DE_ACESSO,Rede de Acesso,RDO-00000000001,Cancelado,NaN
4,4,13093665,3-INSTALAÇÃO_DE_ACESSO,Rede de Acesso,RDO-00000000001,Cancelado,NaN


In [47]:
df_projetos.describe()

,CODCT
count,27418.00000
mean,4.62386
std,1.39646
min,1.00000
25%,4.00000
50%,4.00000
75%,5.00000
max,7.00000


In [48]:
df_escopo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15596 entries, 0 to 15595
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   PROJETO          15596 non-null  int64 
 1   Fase             15596 non-null  object
 2   Região           15596 non-null  object
 3   UF               15596 non-null  object
 4   Município        15596 non-null  object
 5   Escopo_RE        15596 non-null  object
 6   Escopo_RI        15596 non-null  object
 7   Escopo_GS        15596 non-null  object
 8   Energia compart  123 non-null    object
dtypes: int64(1), object(8)
memory usage: 1.1+ MB


In [49]:
df_escopo.head()

,PROJETO,Fase,Região,UF,Município,Escopo_RE,Escopo_RI,Escopo_GS,Energia compart
0,12001058,B,Norte,AC,Cruzeiro do Sul,Sim,Sim,Não,NaN
1,12001953,B,Norte,AC,Mâncio Lima,Sim,Sim,Não,NaN
2,12001996,B,Norte,AC,Mâncio Lima,Sim,Sim,Não,NaN
3,12002003,B,Norte,AC,Mâncio Lima,Sim,Sim,Não,NaN
4,12002054,B,Norte,AC,Mâncio Lima,Sim,Sim,Não,NaN


In [50]:
df_escopo.describe()

,PROJETO
count,1.559600e+04
mean,2.288679e+07
std,9.444828e+06
min,1.100046e+07
25%,1.501865e+07
50%,2.120760e+07
75%,2.906147e+07
max,5.222635e+07
